# 건설자재 품질검사 등록정보

In [52]:
from common import commonFunc as cf
import pandas as pd
import pickle

pd.set_option('display.max_columns', None)
metadata = pd.read_excel("../input/datalake_meta22.xlsx")

SITENAME = "건설사업정보시스템"
DATANAME= "건설자재 품질검사 등록정보"
with open("../input/calsapikey.pickle","rb") as fr:
    APIKEY = pickle.load(fr)

targetData = metadata.loc[metadata.자료명==DATANAME]
preSetFolder = targetData["저장폴더"].values[0]

preSetFolder

'../output/건설사업정보시스템/건설자재 품질검사 등록정보/selectIoPmQtlTsitStsList.csv'

In [53]:
APIKEYLEN = len(APIKEY)

In [54]:
targetData = metadata.loc[metadata.자료명==DATANAME]

##### 파라미터 설정 #####
URL = targetData["URL"].values[0]
SERVICENAME = targetData["서비스키"].values[0]
SERVICENAME = SERVICENAME.split(".")[0]
REQPARAM = targetData["요청변수"].values[0]
REQPARAM = REQPARAM.split(",")
PRIMARYKEY = targetData["기본키"].values[0]

In [55]:
JSONKEY = "items"
DUMMY = 0

In [56]:
#### 페이지번호 설정 ###
PAGEYN=1
if REQPARAM.count("pageyn") == 0:
    PAGEYN = 0
else:
    PAGEYN = 1
PAGEYN    

0

In [57]:
resultDfMerged = pd.DataFrame()

In [58]:
def saveLastSearchYr(paramStartYr):
    cf.saveparam(paramStartYr, SITENAME, DATANAME, SERVICENAME+"_LASTSEARCHYR")
    print("Last Search Yr {} is saved.".format(paramStartYr))
def loadLastSearchYr():
    ret = cf.loadparam(SITENAME, DATANAME, SERVICENAME+"_LASTSEARCHYR")
    print(ret+" is loaded")
    return ret

In [59]:
import datetime

currentDateTime = datetime.datetime.now()
date = currentDateTime.date()
endYr = date.strftime("%Y")
endYr

'2022'

In [67]:
# 기준년도 산정
# 2001년 부터 데이터 존재
startYr = "2001"
try:
    startYr = loadLastSearchYr()
except Exception as e:
    print(e)
# startYr = "2001" # <- 최초 데이터 수집 이후 주석처리해야 함
dateRange = list(map(int,pd.period_range(startYr,endYr,freq='Y').strftime('%Y').tolist()))
dateRange

2021 is loaded


[2021, 2022]

## 건설자재 품질검사 등록정보 전용 스크랩 함수

In [61]:
import requests
from lxml import html
from bs4 import BeautifulSoup 
import pandas as pd
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import os
import time

STDENCODING="utf-8"
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
### 파마리터정의: 
###   - inurl: 메타정보의 "URL"컬럼값 (예: https://www.calspia.go.kr/io/openapi/cm/selectIoCmConstructionList.do )
###   - inSiteName: 메타정보의 "자료대상" (예: 건설사업정보시스템)
###   - inDataName: 메타정보의 "자료명" (예: 공사정보 목록)
###   - inServiceName: 메타정보의 "기본키" (예: serviceKey + Format)
###   - inParam: 메타정보의 "파라미터 정보" (예: 페이지 파라미터 존재 시 1 값")
###   - inPageYn: 메타정보의 "페이지 정보" (예: 페이지 파라미터 존재 시 1 값")
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
def myscrapy(inUrl, inSiteName, inDataName, inServiceName, inParam, inPageYn, jsonkey="items", dummy=0, inType="jsonabnormal"):
    try:
        emptyPd = pd.DataFrame()
        i=1
        while True:
            time.sleep(1.2)
            print("{} page scraping start".format(i))
            
            if(inPageYn==1):
                inParam["pageNo"] = i
            queryParams = '?' + urlencode(inParam)
            requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)
            response = requests.get(inUrl+queryParams,verify=False)
            response.encoding=STDENCODING
            rowData = pd.DataFrame()
            if(inType=="jsonabnormal"):
                # 비정상 데이터는 response 섹션이 없음
                if(response.json().get('response') == None):
                    jsondata = response.json()["header"]["resultMsg"]
                    if( jsondata != "NORMAL_SERVICE"):
                        print("SERVER ERROR ",jsondata)
                        break
                    
#               여기서부터는 데이터는 있는데, totalcount>0 경우는 items 정보에 세부정보가 더 있음
               
                jsondata = response.json()["response"]["body"][jsonkey]
                if( jsondata == []):
                    print("{} page is empty".format(i))
                    break
                if( jsonkey == "detail1"):
                    jsondata["index"]=[0]
                rowData = pd.DataFrame(jsondata)
                emptyPd = emptyPd.append(rowData)

                print(">totalCount "+str(response.json()["response"]["body"]["totalCount"]))
                if(0 < int(response.json()["response"]["body"]["totalCount"])):
                    global g_totalCount
                    if g_totalCount == 0:
                        g_totalCount = int(response.json()["response"]["body"]["totalCount"])
                    
            else:
                print("Error")          
     
            if(inPageYn == 0):
                print("{} no pageNo".format(inPageYn))
                break
            i = i+1

        emptyPd.columns = emptyPd.columns.str.lower()
        emptyPd.shape
#         print("dataframe{}, param:{} rows: {} completed".format(inDataName,inParam, emptyPd.shape[1] )     )
        return emptyPd       
    except Exception as e:
            print(e)     

In [62]:
resultDfMerged = pd.DataFrame()

# 건설자재 품질검사 등록정보 목록 수
g_totalCount = 0

In [63]:
numOfRows = 1000 # 서버에서 최대 응답 가능한 row임
resultDf = pd.DataFrame()
print("len of dateRange : ",len(dateRange))
for i in range(0,len(dateRange)):
    pageNo = 1
    print("i=",i)
    year = dateRange[i]
    apiNo = i%APIKEYLEN
    g_totalCount = 0
    
    while True:
        BASEPARAM={"serviceKey":APIKEY[apiNo],"year": year, "sortField": "tstBgDt","sortOrder": "ASC", "pageNo": pageNo, 'numOfRows':numOfRows, "type":"json"}
        print("BASEPARAM ",BASEPARAM)
        resultDf = myscrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, JSONKEY, DUMMY,"jsonabnormal")
        if resultDf.empty: # 정상 데이터가 없는 경우
            print("No Data")
            break
        else:
            resultDfMerged = resultDfMerged.append(resultDf)
            g_totalCount -= numOfRows
            print("g_totalCount : ", g_totalCount)
            if(g_totalCount <= 0):
                break
            pageNo += 1

len of dateRange :  22
i= 0
BASEPARAM  {'serviceKey': '0974320D-C689-4543-8464-10DE1E5505A9', 'year': 2001, 'sortField': 'tstBgDt', 'sortOrder': 'ASC', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start


KeyboardInterrupt: 

In [64]:
cf.savedata(resultDfMerged, SITENAME,DATANAME,SERVICENAME)

건설자재 품질검사 등록정보 save compled


In [65]:
resultDfMerged.shape

(0, 0)

### 다음 회차 수행을 위해 현재 검색종료 Year를 저장한다. 
### 다음 회차 검색은 현재 저장한 검색종료 Year가 시작점이 된다. 

In [66]:
# 1년 차감하여 저장, 당해 년은 계속 데이터가 쌓이기 때문
saveLastSearchYr(str(int(endYr)-1))

Last Search Yr 2021 is saved.
